# Firefly Demo

## Imports from the stack, firefly, and standard libraries.

In [ ]:
from lsst.daf.persistence import Butler
import lsst.afw.display as afw_display

from firefly_client import FireflyClient

from urllib.parse import urlparse, urlunparse, ParseResult
from IPython.display import display, Markdown
import os

## Get some test data.

These data were downloaded from: [here](https://lsst-web.ncsa.illinois.edu/~krughoff/data/twinkles_subset.tar.gz).  See the `README.txt` in `/project/shared/data/Twinkles_subset` for more info.

In [ ]:
butler = Butler('/project/shared/data/Twinkles_subset/output_data_v2')
dataId = {'filter': 'r', 'raft': '2,2', 'sensor': '1,1', 'visit': 235}

Retrieve an image and a catalog from the butler.

In [ ]:
calexp = butler.get('calexp', **dataId)
src = butler.get('src', **dataId)

## Using `afwDisplay`

Set up the display for using `afwDisplay`.

In [ ]:
afw_display.setDefaultBackend('firefly')
my_channel = '{}_test_channel'.format(os.environ['USER'])
internal_server = 'http://firefly:8080'

> Currently, the interaction URL and the visualization URL are different since the interaction URL is internal to the k8s cluster.  This will likely change, but for now we will discover the name to use for the visualization URL.

In [ ]:
parsed_url = urlparse(os.environ['EXTERNAL_URL'])
parse_dict = parsed_url._asdict()
parse_dict['path'] = ''
parsed_url = ParseResult(**parse_dict)
external_server = urlunparse(parsed_url)

In [ ]:
display(Markdown('> Open a browser window to [this]({}/firefly/slate.html?__wsch={}) link.'.format(external_server, my_channel)))

Set up the `afwDisplay.Display` object to use later.

In [ ]:
afw_display = afw_display.getDisplay(frame=1, 
                                  host=internal_server,
                                  name=my_channel)

First display an image with mask planes and then overplot some sources.  This is limited to the top 500 as plotting the whole catalog takes some time, since it is a serial process.

In [ ]:
afw_display.mtv(calexp)

In [ ]:
for record in src[:500]:
    afw_display.dot('o', record.getX(), record.getY())

## Using `FireflyClient` directly.

We can also use the firefly client directly to make plots and add catalogs to the visualization.  First construct the `FireflyClient` object.

In [ ]:
host = internal_server
html = "slate.html"
fc = FireflyClient(host, channel=my_channel, html_file=html)

We need a FITS file to pass to Firefly, so first persist the catalog to local storage.  Then upload it.

In [ ]:
src.writeFits('src.fits')
src_key = fc.upload_file('src.fits')

Once the file is uploaded, we can generate a scrollable table view widget.  Note that the label you give the table here, `tbl_id='src'`, is how you will refer to that dataset in future plots.

In [ ]:
status = fc.show_table(src_key, tbl_id='src', page_size=1000)

Finally, we can make a scatter plot and send it to the display canvas.  Plots are rendered by plotly, so follow the same syntax for construction.  For info on plotly see the [primer](https://plot.ly/python/getting-started/) and [examples](https://plot.ly/python/line-and-scatter/).

Style for the plot

In [ ]:
layout1 = dict(
               title='test ap flux/model mag vs. log(ap flux)', 
               xaxis=dict(title='Model'), 
               yaxis=dict(title='Ap/Model')
              )

Data to plot.  Note the reference to 'src'.  That allows for columnar math in the construction of the plot.

In [ ]:
trace1 = dict(
              tbl_id='src', 
              x='tables::base_CircularApertureFlux_12_0_flux/base_GaussianFlux_flux',
              y='tables::log10(base_CircularApertureFlux_12_0_flux)',
              mode='markers',
              type='scatter',
              marker=dict(size=4)
             )

Send the plot to the visualization canvas.

In [ ]:
status = fc.show_chart(layout=layout1, data=[trace1])